In [41]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [42]:
filepath = tf.keras.utils.get_file('shakespeare.txt', './data/shakespeare.txt')

In [43]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [44]:
text = text[100000:1000000]

In [45]:
characters = sorted(set(text))

In [46]:
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [129]:
SEQ_LENGTH = 48
STEP_SIZE = 3

In [130]:
sentences = []
next_characters = []

In [131]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i: i + SEQ_LENGTH])
  next_characters.append(text[i + SEQ_LENGTH])

In [132]:
X = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
Y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)

In [133]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    X[i, t, char_to_index[character]] = 1
  Y[i, char_to_index[next_characters[i]]] = 1

In [137]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
csv_logger = tf.keras.callbacks.CSVLogger('./training_log/log.csv', separator=',', append=True)

model.fit(X, Y, batch_size=256, epochs=20, callbacks=[csv_logger])

Epoch 1/20
1172/1172 [==============================] - 86s 72ms/step - loss: 1.9272
Epoch 2/20
1172/1172 [==============================] - 85s 73ms/step - loss: 1.5807
Epoch 3/20
1172/1172 [==============================] - 86s 73ms/step - loss: 1.4987
Epoch 4/20
1172/1172 [==============================] - 86s 74ms/step - loss: 1.4564
Epoch 5/20
1172/1172 [==============================] - 88s 75ms/step - loss: 1.4283
Epoch 6/20
1172/1172 [==============================] - 88s 75ms/step - loss: 1.4083
Epoch 7/20
1172/1172 [==============================] - 88s 75ms/step - loss: 1.3933
Epoch 8/20
1172/1172 [==============================] - 88s 75ms/step - loss: 1.3801
Epoch 9/20
1172/1172 [==============================] - 89s 76ms/step - loss: 1.3707
Epoch 10/20
1172/1172 [==============================] - 88s 75ms/step - loss: 1.3625
Epoch 11/20
1172/1172 [==============================] - 89s 76ms/step - loss: 1.3562
Epoch 12/20
1172/1172 [==============================] - 88s 75

In [138]:
model.save('textgenerator.model')

INFO:tensorflow:Assets written to: textgenerator.model\assets


INFO:tensorflow:Assets written to: textgenerator.model\assets


In [139]:
model = tf.keras.models.load_model('textgenerator.model')

In [140]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [141]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence
  for i in range(length):
    A = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(sentence):
      A[0, t, char_to_index[character]] = 1
    
    predictions = model.predict(A, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character
  
  return generated

In [142]:
print('-----------------------------0.2-------------------------------')
print(generate_text(256, 0.2))
print('-----------------------------0.4-------------------------------')
print(generate_text(256, 0.4))
print('-----------------------------0.6-------------------------------')
print(generate_text(256, 0.6))
print('-----------------------------0.8-------------------------------')
print(generate_text(256, 0.8))
print('------------------------------1--------------------------------')
print(generate_text(256, 1))

-----------------------------0.2-------------------------------
sh'd, does exceed you all.

brutus:
well, well, the first than the count thou art thoughts,
that i have send the world that i have send
his son than the bed to the blood to the
lord angelo's death, and the great the prince.

romeo:
i will not see the best the country's death.

pompey:
what shall i the b
-----------------------------0.4-------------------------------
nd we hear
such goodness of your justice, that one send
him in the offers she hath been for my conding.

gloucester:
which is the poison of the sister of the crown,
that thou art thou art to her orance and the sens
in the conceit to the prince's strain'd conclany
that which you have still not be not to 
-----------------------------0.6-------------------------------
s: guess, but by my entertainment
with him, if thou wilt diserves the crown.

leontes:
this is the brother, if thou art true field.

provost:
ay, if thou art though and here to land of face,
and the

C:\Users\glori\AppData\Local\Temp\ipykernel_18336\388290196.py:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds)/temperature


but lusty, young, and cheerly drawing breath.
look her partice in with dave must seeming
that be not go leave and send and fellow,
from a montages of my heart. you know not hear:
and you have king by the men for pleasene.

duke vincentio:
look her desain's in good that shall she should be-gone
matter bl
------------------------------1--------------------------------
eps in capel's monument,
and her immortal part with the tring tratwal vows.

isabella:

hortess:
syering himself.

usbraal:
in scoonge to make thy face?

provost:
for well, in thy of at what man,
and i'll through imprison from me, there believed
hath undoning so is store to ever his york?

vols:
that wa
